In [31]:
import sys
sys.path.insert(0, "../../../")
from clases.utils import setup_environment, connect_database, sys_anio, sys_mes_ini, sys_mes_fin

setup_environment()
conn = connect_database()
anio=sys_anio()
mes_ini=sys_mes_ini()
mes_fin=sys_mes_fin()


Conexion exitosa con la Base de datos:irvin_hisminsa


In [32]:
def gestante_anemia(conn,anio,mes_ini,mes_fin):
    gestante_anemia=conn.df(f"""
 DROP TABLE IF EXISTS his_minsa;  
  
SELECT nt.id_cita
,provincia
    ,    distrito
    ,      red
    ,    microred
    ,    nombre_eess
    ,    anio
    ,    mes
    ,    codigo_item
    ,    numero_documento
    ,    cod_eess
    ,    fecha_atencion
    ,nt.tipo_diagnostico
 FROM 
    maestros.nominal_trama nt
INNER JOIN maestros.maestro_paciente mp ON
    nt.id_paciente = mp.id_paciente
INNER JOIN maestros.eess_geresa_cusco egc ON
    nt.id_establecimiento = egc.id_eess
WHERE 
    nt.codigo_item IN (
                    'Z3491', 'Z3591', 'Z3492', 'Z3592', 'Z3493', 'Z3593',    -- Atencion PreNatal.
                    '85018', '85018.01',    -- Dosaje de Hb 
                    'O990',    -- Dx Anemia.
                    '99199.26', '59401.04'    -- Tratamiento Dx Anemia.
                )
    AND genero = 'F'
    AND anio IN (2024)
    AND mes BETWEEN 1 AND 2
    AND id_tipo_documento = '1'
    AND cat IN ('I-1', 'I-2', 'I-3', 'I-4')
    AND egc.cod_ue != 0    --ESSALUD
 """)
    return gestante_anemia

In [33]:
his_minsa=gestante_anemia(conn,anio,mes_ini,mes_fin)

d:\www\Python-GERESA\modulos\gestante\fed\../../..\clases\bd\conexion2.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [34]:
# 1. Mujeres gestantes con atención prenatal en cualquier momento de la gestación
den_apn = his_minsa.loc[his_minsa['codigo_item'].isin(['Z3491', 'Z3591', 'Z3492', 'Z3592', 'Z3493', 'Z3593']) & 
                        (his_minsa['anio'] == 2024), 
                        ['numero_documento', 'cod_eess']]



In [35]:
# 2. Mujeres gestantes con Dosaje de HB o Perfil Obstétrico
den_hb = den_apn.merge(
                  his_minsa.loc[(his_minsa['codigo_item'].isin(['85018', '85018.01'])) & 
                                (his_minsa['tipo_diagnostico'] == 'D') & 
                                (his_minsa['anio'] == 2024), 
                                ['mes', 'cod_eess', 'numero_documento', 'fecha_atencion', 'id_cita']], 
                  on=['numero_documento', 'cod_eess'], 
                  how='inner')
den_hb =den_hb.drop_duplicates()
den_hb['año'] = 2024
den_hb['den_hb'] = 1



In [36]:
# 3. Mujeres gestantes con Dx de Anemia
den = den_hb.merge( 
               his_minsa.loc[(his_minsa['codigo_item'] == 'O990') & 
                             (his_minsa['tipo_diagnostico'] == 'D'), 
                             ['fecha_atencion', 'cod_eess', 'numero_documento', 'id_cita']], 
               on=['numero_documento', 'cod_eess', 'id_cita'], 
               how='inner')
den =den.drop_duplicates()
den['año'] = 2024
den['den'] = 1

print(den)

    numero_documento  cod_eess  mes fecha_atencion_x    id_cita   año  den_hb fecha_atencion_y  den
0           76685854      2366    2       2024-02-01  988196102  2024       1       2024-02-01    1
1           71794720     12932    1       2024-01-01  979429028  2024       1       2024-01-01    1
2           47482022      2315    1       2024-01-16  980085626  2024       1       2024-01-16    1
3           75673580      2303    1       2024-01-09  977264162  2024       1       2024-01-09    1
4           77244175      2300    1       2024-01-02  975730833  2024       1       2024-01-02    1
..               ...       ...  ...              ...        ...   ...     ...              ...  ...
477         43666654      2300    2       2024-02-21  994083821  2024       1       2024-02-21    1
478         71789090      2300    2       2024-02-21  994401978  2024       1       2024-02-21    1
479         71854581      2315    2       2024-02-23  994876889  2024       1       2024-02-23    1


In [ ]:
"""      
  -- 1.Mujeres gestantes con atencion prenatal en cualquier momento de la gestacion.
CREATE TEMP TABLE den_apn AS
SELECT DISTINCT numero_documento, cod_eess
FROM his_minsa a
WHERE codigo_item IN ('Z3491', 'Z3591', 'Z3492', 'Z3592', 'Z3493', 'Z3593')
      --AND EXTRACT(MONTH FROM fecha_atencion) = 1
      AND EXTRACT(YEAR FROM fecha_atencion) = 2024;
  
  -- 2. Mujeres gestantes (1.) con Dosaje de HB o Perfil Obstetrico.
  CREATE TEMP TABLE den_hb AS
SELECT DISTINCT
       2024 AS año,
       b.mes,
       a.cod_eess,
       a.numero_documento,
       b.fecha_atencion AS fecha_hb,
       b.id_cita AS id_cita_hb,
       1 AS den_hb
FROM den_apn a
INNER JOIN his_minsa b ON a.numero_documento = b.numero_documento AND a.cod_eess = b.cod_eess
WHERE b.codigo_item IN ('85018', '85018.01') AND b.tipo_diagnostico = 'D'
      --AND EXTRACT(MONTH FROM b.fecha_atencion) = @mes_inicio
      AND EXTRACT(YEAR FROM b.fecha_atencion) = 2024;
  
  
-- 3. Mujeres gestantes (2.) con Dx de Anemia.
CREATE TEMP TABLE den AS
SELECT DISTINCT
       a.año,
       a.mes,
       a.cod_eess,
       a.numero_documento,
       b.fecha_atencion AS fecha_dx,
       1 AS den
FROM den_hb a
INNER JOIN his_minsa b ON a.numero_documento = b.numero_documento AND a.cod_eess = b.cod_eess AND a.id_cita_hb = b.id_cita
WHERE b.codigo_item = 'O990' AND b.tipo_diagnostico = 'D';"""